In [1]:
import cv2
import os
from random import shuffle
from tqdm import tqdm
import numpy as np



In [2]:
#pip install tflearn
TRAIN_DIR="C:/Users/Lab009/Desktop/dataset/train"
TEST_DIR ="C:/Users/Lab009/Desktop/dataset/test"



def label_img(img):
    word_label = img.split('.')[0]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [0,1]
    #                             [no cat, very dog]
    elif word_label == 'dog': return [1,0]


In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (50,50))
        training_data.append([np.array(img),np.array(label)])
        
    shuffle(training_data)
    #np.save('train_data.npy', training_data)
    return training_data


In [4]:
train_data = create_train_data()

100%|██████████| 25000/25000 [00:50<00:00, 495.16it/s]


In [5]:
#conda install tflearn or pip install tflearn
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,fully_connected,dropout
from tflearn.layers.estimator import regression


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model=input_data(shape=[None,50,50,1], name='input')


# In[8]:


#first convolution Layer
model=conv_2d(model,46,5,activation='relu')
model=max_pool_2d(model,5)

#second Convolution Layer
model=conv_2d(model,42,5,activation='relu')
model=max_pool_2d(model,5)

#third Convolution Layer
model=conv_2d(model,38,5,activation='relu')
model=max_pool_2d(model,5)

#fully Connected Layer
model=fully_connected(model,1024,activation='relu')

model=dropout(model,0.7)
model=fully_connected(model,2,activation='softmax')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model=regression(model,
                optimizer='adam',
                loss='categorical_crossentropy',
                learning_rate=0.003,
                name='output')



Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
cnnmodel=tflearn.DNN(model)

Instructions for updating:
Use tf.cast instead.


In [9]:
train=train_data[:-500]  #24500 for training
test=train_data[-500:]   #500 for testing

In [10]:
X=np.array([i[0] for i in train]).reshape(-1,50,50,1)
Y=[i[1] for i in train] 


In [11]:

test_x=np.array([i[0] for i in test]).reshape(-1,50,50,1)
test_y=[i[1] for i in test] 


In [12]:
cnnmodel.fit({'input':X},{'output':Y}, n_epoch=10,
            validation_set=({'input':test_x},{'output':test_y}),
            show_metric=True)

Training Step: 3829  | total loss: 11.93638 | time: 89.057s
| Adam | epoch: 010 | loss: 11.93638 - acc: 0.4816 -- iter: 24448/24500
Training Step: 3830  | total loss: 11.93001 | time: 90.307s
| Adam | epoch: 010 | loss: 11.93001 - acc: 0.4819 | val_loss: 11.05241 - val_acc: 0.5200 -- iter: 24500/24500
--
